In [56]:
import loading as ld
import time
import sys
import load_data as lo
import similarities as sm
import pandas as pd
class CollaborativeRecommender:
    def __init__(self,urls = ['Books.csv','Users.csv','Book-Ratings.csv']):
        self.urls = urls
        self.sim = sm.Similarity()
        self.dataset = lo.Load_data(self.urls)
        self.data_frame = None
        self.user_avg_rating_df = None
        self.user_rating_counts_df  = None
        self.user_rating_std_df = None
        self.closest_n_similarity_df = None
    def initate(self):
        loading = ld.Loading()
        print("## Collabarative Recomondation System by Asiri Thenuka Thenabadu : 31048197 ##")
        loading.start(text_start="Program Starting")
        time.sleep(1)
        loading.stop(text_end= "")
        loading.start(text_start="Loading Data Sets")
#         dataset = lo.Load_data(self.urls)
        self.set_merged_dataset_df()
#         self.data_frame = self.get_merged_dataset()
        self.data_frame.head()
        loading.stop(text_end= "Data Set loaded Sucessfully")
#         loading.start(text_start="Loading Similarities")
#         sim = sm.Similarity()
#         loading.stop(text_end= "Similarities loaded Sucessfully !")
        loading.start(text_start="Please Insert Value to Begin ")
        time.sleep(1)
        loading.stop(text_end= "")
        param1 = int(input("Target User Id : "))
        self.set_target_user(param1)
        self.get_target_user()
        
        loading.start(text_start="Please Select Similarity matrix ")
        time.sleep(1)
        loading.stop(text_end= "")
        awailable_modals = sm.awailable_modules()
        for i,val in enumerate(awailable_modals):
            print("\r"+str(i+1)+"."+val+" \n")
        sim_matrix = -99
        while (int(sim_matrix) <= 0 or int(sim_matrix) > len(awailable_modals) or isinstance(sim_matrix, int) == False):
            sim_matrix = int(input("Similarity Matrix Id : "))
            if(isinstance(sim_matrix, int)):
                if(int(sim_matrix) < 0 and int(sim_matrix) > len(awailable_modals)):
                    loading.start(text_start="Invalid Similarity Please select Correct Similarity ")
                    time.sleep(2)
                    loading.stop(text_end= "")
            else:
                loading.start(text_start="Invalid Similarity Please select Correct Similarity ")
                time.sleep(2)
                loading.stop(text_end= "")
#                 sim_matrix = -99
        else:
            pass
        print("You selected, "+awailable_modals[sim_matrix-1])
        cosest_n = self.get_closest_n_users(awailable_modals[sim_matrix-1])
        print(cosest_n)
#         self.get_closest_n_users(sim_matrix)
        
    def get_books_data(self):
        return self.dataset.get_books_data()
    def get_user_data(self):
        return self.dataset.get_user_data()
    def get_rating_data(self):
        return self.dataset.get_rating_data()
    def get_merged_dataset(self):
        return self.dataset.merge_dataset()
    def set_merged_dataset_df(self):
        self.data_frame = self.get_merged_dataset()
    def get_merged_dataset_df(self):
        return self.data_frame
    def plot_rating_distribution(self):
        return self.data_frame['bookRating'].hist()
    def get_users_avg_rating_DF():
        self.user_avg_rating_df = self.data_frame.groupby('userID')["bookRating"].mean().rename("user_avg_rating").reset_index()
    def get_user_rating_counts_DF():
        self.user_rating_counts_df = self.data_frame.groupby('userID')["bookRating"].count().rename("user_rating_counts").reset_index()
    def get_user_rating_std_DF():
        self.user_rating_std_df = self.data_frame.groupby('userID')["bookRating"].std(ddof=0).rename("user_rating_std").reset_index()
    def get_target_user(self):
#         self.data_frame['userID'].count()
        return self.data_frame.loc[self.data_frame['userID'] == self.target_user_id][['bookRating','ISBN']]
    def set_target_user(self,target_user_id):
        self.target_user_id = target_user_id
        self.target_user_df = self.get_target_user()
    def get_related_users(self):
#         print("self.target_user_id =======>",self.target_user_id)
        temp_df = self.data_frame[self.data_frame['ISBN'].isin(self.target_user_df['ISBN'].tolist())]
        return temp_df.loc[temp_df['userID'] != self.target_user_id]
    def set_related_users(self):
        self.related_users = self.get_related_users()
#         print("set_related_users inside ===!! ",self.related_users)
        self.related_users
    def find_related_users(self):
#         print("find_related_users func")
        #Getting related Users to the target user
        self.set_related_users()
#         print("set_related_users ===",self.related_users)
        # Grouping related Users
        temp_group_by_df = self.related_users.groupby(['userID'])
        #Sorting it so users with books most in common with the target user will have priority
        self.related_users_orderd = sorted(temp_group_by_df,  key=lambda x: len(x[1]), reverse=True)
    def set_closest_n_users(self):
        simDict = {}
        for name,group in self.related_users_orderd:
            #Let's start by sorting the input and current user group so the values aren't mixed up later on
            group = group.sort_values(by='ISBN')
            targetsBooks = self.target_user_df.sort_values(by='ISBN')
            #Get the rating scores for the books that they both have in common
            temp_df = targetsBooks[targetsBooks['ISBN'].isin(group['ISBN'].tolist())]
            tempRatingList = temp_df['bookRating'].tolist()
            tempGroupList = group['bookRating'].tolist()
            if(len(tempRatingList) > 2):
                if(sum(tempRatingList) > 0 and sum(tempGroupList) > 0):
                    similarity = sm.getSimilarity(sim = self.similarity_algo, target_r =tempRatingList , closest_r = tempGroupList)
                    if (similarity != 'Undifined'):
                        simDict[name] = similarity
                    else:
                        pass
                else:
                    pass
            else:
                pass
        temp_sim_df = pd.DataFrame.from_dict(simDict, orient='index')
        temp_sim_df.columns = ['similarityIndex']
        temp_sim_df['userID'] = temp_sim_df.index
        temp_sim_df.index = range(len(temp_sim_df))
        self.closest_n_similarity_df = temp_sim_df.sort_values(by='similarityIndex', ascending=False)[0:50]
    def get_closest_n_users(self,similarity_algo):
#         print("here")
        self.similarity_algo = similarity_algo
        self.find_related_users()
        self.set_closest_n_users()
        return self.closest_n_similarity_df
        
        
# def main():
#     initate()
    
    
    
    

In [57]:
def main():
    rec = CollaborativeRecommender()
    rec.initate()
#     276704

In [58]:
main()

## Collabarative Recomondation System by Asiri Thenuka Thenabadu : 31048197 ##
Target User Id : 276704
1.cosine_similarity rity matrix     

2.euclidean_similarity 

3.hamming_distance_similarity 

4.minkowski_distance 

5.pearson_correlation 

6.pearson_correlation_scipy 

7.spearman_correlation_similarity 

8.squared_euclidean_similarity 

Similarity Matrix Id : 6
You selected, pearson_correlation_scipy
   similarityIndex  userID
9         1.000000  236959
1         0.687500  185233
5         0.609994  170513
0         0.411706   11676
8        -0.295745  236283
6        -0.500000  225087
7        -0.500000  235392
2        -0.500000   13552
3        -0.500000   16795
4        -0.991241   43842


In [13]:
import similarities as sm

sim = sm.Similarity()

In [15]:
sm.awailable_modules()

['cosine_similarity',
 'euclidean_similarity',
 'hamming_distance_similarity',
 'minkowski_distance',
 'pearson_correlation',
 'pearson_correlation_scipy',
 'spearman_correlation_similarity',
 'squared_euclidean_similarity']

In [3]:
# c.initate()

## Collabarative Recomondation System by Asiri Thenuka Thenabadu : 31048197 ##
Data Set loaded Sucessfully

In [13]:
# sim_matrix = -99
# (sim_matrix < 0 or sim_matrix > 6)

True

In [52]:
sim = CollaborativeRecommender()
sim.set_merged_dataset_df()
sim.set_target_user(276704)
sim.target_user_df
ddf = sim.get_closest_n_users('pearson_correlation_scipy')
ddf

,similarityIndex,userID
9,1.000000,236959
1,0.687500,185233
5,0.609994,170513
0,0.411706,11676
8,-0.295745,236283
6,-0.500000,225087
7,-0.500000,235392
2,-0.500000,13552
3,-0.500000,16795
4,-0.991241,43842


In [50]:
ddf